In [ ]:

import cv2
import matplotlib.pyplot as plt
import numpy as np

from ai_trainer.models import BlazePoseModel
from ai_trainer.drawing import *
from ai_trainer.feedback.lunges import give_feedback

BLAZEPOSE_MODEL_PATH = "./models/blazepose_full.onnx"
blazepose_model = BlazePoseModel(model_path=BLAZEPOSE_MODEL_PATH)

IMAGE_PATH = "./assets/8.png"  # change if needed

# read image
image = cv2.imread(IMAGE_PATH)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# copy image to plot on it
plot_image = image.copy()
img_h, img_w = image.shape[:2]
# print(img_h, img_w)

kps = blazepose_model.predict([image])[0]  # batch size is 1
# print(kps[24])
# print(kps)
probs = kps.T[3]

if not all(probs<0):
    # denormalize keypoints:
    x, y, z = kps.T[:3]
    # x_img = x * img_w
    # y_img = y * img_h
    pose_3d = np.column_stack((x, y, z))
    pose_2d = np.column_stack((x, y))
    
    # plot keypoints on image
    sample = draw_pose(
        image=plot_image,
        keypoints=pose_2d,
        disposition="mediapipe", # blazepose keypoints are in mediapipe format
        thickness=2,
    )

    feedback, possible_corrections = give_feedback(pose_3d)
    number_corrections = len(list(feedback.keys())[1::])

    y_text_pos1 = 0
    y_text_pos2 = 0
    for correction in possible_corrections:
        y_text_pos1+=25
        y_text_pos2+=35
        if correction in list(feedback.keys()):
            sample = draw_text_with_border(
                image=sample,
                text=feedback[correction],
                origin=(10, 70),
                font_scale=0.8,
                font_face = cv2.FONT_HERSHEY_SIMPLEX,
                color_in=(250, 50, 50),
                thickness_in=2,
                color_out= (0, 0, 0),
                thickness_out = 3,
     
            )
            sample = draw_text_with_border(
                image=sample,
                text=feedback[correction],
                origin=(10, 120),
                font_scale=0.8,
                font_face = cv2.FONT_HERSHEY_SIMPLEX,
                color_in=(250, 50, 50),
                thickness_in=2,
                color_out= (0, 0, 0),
                thickness_out = 3,
     
            )
            hip_coord = kps[23]
            # draw_dotted_line(sample, hip_coord, start=hip_coord[1]-80, end=hip_coord[1]+20, line_color=(0, 127, 255))
plt.figure(figsize=(5, 5))
plt.imshow(plot_image)

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('models/yolo/best.pt')  # pretrained YOLOv8n model

# Run batched inference on a list of images
results = model(['assets/8.png', 'assets/8.png'], stream=True)  # return a generator of Results objects

# Process results generator
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    # result.show()  # display to screen
    print("1 +", keypoints)
    # result.save(filename='result.jpg')  # save to disk

In [2]:
pip install ultralytics

^C
Note: you may need to restart the kernel to use updated packages.


In [4]:
from ultralytics import YOLO

# Load a model
model = YOLO('../models/yolo3/best.pt')  # pretrained YOLOv8n model

# Predict with the model
results = model.predict('https://ultralytics.com/images/bus.jpg') 

# Extract keypoint
result_keypoint1 = results[0].keypoints.data.cpu().numpy()[0]
result_keypoint = results[0].keypoints.xyn.cpu().numpy()[0]
# keypoint_data = results[0].cpu().detach().numpy()
print(result_keypoint1)
print(result_keypoint)


Found https://ultralytics.com/images/bus.jpg locally at bus.jpg


AttributeError: 'Pose' object has no attribute 'detect'